In [9]:
%%capture
%run 'Plotting.ipynb'

import itertools
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

from scipy import interp

import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

In [2]:
# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Run classifier, using a model that is too regularized (C too low) to see
# the impact on the results
classifier = svm.SVC(kernel='linear', C=0.01)
y_pred = classifier.fit(X_train, y_train).predict(X_test)

In [3]:
def plot_binary_confusion_matrix():
    cm = [[1,2],[3,4]]
    cm_text = [['False Negative','True Negative'],['True Positive','False Positive']]
    x = ['Class A','Class B']
    y = ['Class B','Class A']
    fig = ff.create_annotated_heatmap(cm, 
                                      x=x, 
                                      y=y, 
                                      annotation_text=cm_text, 
                                      colorscale='Viridis')
    
    fig['layout']['yaxis'].update({'title':'True Value'})
    fig['layout']['xaxis'].update({'title':'Predicted Value'})
    iplot(fig)

In [4]:
plot_binary_confusion_matrix()

True Positive (TP) - same as "hit"

True Negative (TN) - same as "correct rejection"

False Positive (FP) - same as "false alarm", Type I error

False Negative (FN) - same as miss, Type II error

True Positive Rate (TPR), Sensitivity, Recall -  
$${\displaystyle \mathrm {TPR} = {\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FN} }}}$$  
True negative rate (TNR) or Specificity
$${\displaystyle \mathrm {TNR} ={\frac {\mathrm {TN} }{\mathrm {TN} +\mathrm {FP} }}}$$

Positive Predictive Value (PPV) or Precision 
$${\displaystyle \mathrm {PPV} ={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FP} }}}$$

Negative Predictive Value (NPV)
$${\displaystyle \mathrm {NPV} ={\frac {\mathrm {TN} }{\mathrm {TN} +\mathrm {FN} }}}$$

False Negative Rate (FNR) or Miss Rate
$${\displaystyle \mathrm {FNR} ={\frac {\mathrm {FN} }{\mathrm {FN} +\mathrm {TP} }}=1-\mathrm {TPR} }$$ 

False Positive Rate (FPR) or Fall-out
$${\displaystyle \mathrm {FPR} ={\frac {\mathrm {FP} }{\mathrm {FP} +\mathrm {TN} }}=1-\mathrm {TNR} }$$

False Discovery Rate (FDR)
$${\displaystyle \mathrm {FDR} ={\frac {\mathrm {FP} }{\mathrm {FP} +\mathrm {TP} }}=1-\mathrm {PPV} }$$

False Omission Rate (FOR)
$${\displaystyle \mathrm {FOR} ={\frac {\mathrm {FN} }{\mathrm {FN} +\mathrm {TN} }}=1-\mathrm {NPV} }$$ 

Accuracy (ACC)
$${\displaystyle \mathrm {ACC} = {\frac {\mathrm {TP} +\mathrm {TN} }{\mathrm {TP} +\mathrm {TN} +\mathrm {FP} +\mathrm {FN} }}}$$ 

F1 score is the harmonic mean of precision and sensitivity
$${\displaystyle F_{1} = 2\cdot {\frac {\mathrm {PPV} \cdot \mathrm {TPR} }{\mathrm {PPV} +\mathrm {TPR} }}={\frac {2\mathrm {TP} }{2\mathrm {TP} +\mathrm {FP} +\mathrm {FN} }}} $$


In [6]:
c_matrix = confusion_matrix(y_test, y_pred)

In [10]:
cm_plot_handler = CMPlotHandler(confusion_matrix=c_matrix,classes=class_names)
cm_plot_handler.plot()

In [38]:
# Binarize the output
y_one_hot = label_binarize(y, classes=[0, 1, 2])
n_classes = y_one_hot.shape[1]

# Add noisy features to make the problem harder
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=.5,
                                                    random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=random_state))

y_score = classifier.fit(X_train, y_train).decision_function(X_test)

In [127]:
roc_plot_handler = MultipleClassROCPlotHandler(y_score,y_test,class_names)
roc_plot_handler.plot()